# Connect to Local Spark Cluster using Spark Context (running via Docker)

In [ ]:
%set_env JAVA_HOME=/Users/akshayiyer/Library/Java/JavaVirtualMachines/jdk8u222-b10/Contents/Home

In [ ]:
%env

In [ ]:
# https://elmiko.github.io/2018/08/05/attaching-notebooks-with-radanalytics.html
import pyspark
conf=pyspark.SparkConf().setMaster('spark://127.0.0.1:7077')
#conf=pyspark.SparkConf().setMaster('local')
sc = pyspark.SparkContext(conf=conf)

In [ ]:
sc

## Program to calculate Pi on Spark Cluster

In [ ]:
%%time
import random
num_samples = 100000000

def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1

count = sc.parallelize(range(0, num_samples)).filter(inside).count()
pi = 4 * count / num_samples

print(pi)

# Connect to Local Spark Cluster using Spark Session (running via Docker)

In [1]:
%set_env JAVA_HOME=/Users/akshayiyer/Library/Java/JavaVirtualMachines/jdk8u222-b10/Contents/Home

env: JAVA_HOME=/Users/akshayiyer/Library/Java/JavaVirtualMachines/jdk8u222-b10/Contents/Home


In [2]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format

In [3]:
config = configparser.ConfigParser()
config.read('/Users/akshayiyer/Dev/GitHub/udacity-dend/udacity-dend-capstone-etl/.aws/access_keys.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config.get('AWS','KEY')
os.environ['AWS_SECRET_ACCESS_KEY']=config.get('AWS','SECRET')

In [4]:
def create_spark_session():
    spark = SparkSession \
        .builder \
        .master('spark://127.0.0.1:7077') \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0")\
        .getOrCreate() 
    return spark

In [5]:
spark = create_spark_session()

## Read JSON file into Spark dataframe 

In [ ]:
song_data_file_path = 's3a://udacity-dend/song_data/A/B/C/TRABCEI128F424C983.json'

song_df = spark.read.json(song_data_file_path)

In [ ]:
song_df.printSchema()